In [5]:
import json
import re

# Open the JSON file and load the data
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/qa_format_output3.json', 'r') as f:
    data = json.load(f)

# Replace "\p" with "\\" to match a literal backslash
regex = r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'

# Apply the regex to each string value in the JSON data
for i in range(len(data)):
    if isinstance(data[i], str):
        data[i] = re.sub(regex, "", data[i])
    elif isinstance(data[i], dict):
        for key in data[i]:
            if isinstance(data[i][key], str):
                data[i][key] = re.sub(regex, "", data[i][key])

# Save the cleaned data back to the same JSON file
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/qa_format_output3.json', 'w') as f:
    json.dump(data, f, indent=2)


In [6]:
for i in range(len(data)):
    if isinstance(data[i], str):
        # Replace all occurrences of \n with an empty string
        data[i] = data[i].replace('\n', '')
    elif isinstance(data[i], dict):
        for key in data[i]:
            if isinstance(data[i][key], str):
                # Replace all occurrences of \n with an empty string
                data[i][key] = data[i][key].replace('\n', '')


In [7]:
import json

# Load the data from the JSON file
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/qa_format_output3.json', 'r') as f:
    data = json.load(f)

# Remove all occurrences of \n from string values in the data
for i in range(len(data)):
    if isinstance(data[i], str):
        data[i] = data[i].replace('\n', '')
    elif isinstance(data[i], dict):
        for key in data[i]:
            if isinstance(data[i][key], str):
                data[i][key] = data[i][key].replace('\n', '')

# Save the modified data back to the same JSON file
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/qa_format_output3.json', 'w') as f:
    json.dump(data, f, indent=2)


In [9]:
import os
import unicodedata

# specify the directory path to the text files
dir_path = "/home/ssarrouf/Documents/test_data_2"

# loop through each file in the directory
for filename in os.listdir(dir_path):
    if filename.endswith(".txt"):
        # read the file contents
        with open(os.path.join(dir_path, filename), "r") as f:
            text = f.read()
        # remove unicode characters
        text = text.replace('\n', '')
        text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
        # overwrite the file with the cleaned text
        with open(os.path.join(dir_path, filename), "w") as f:
            f.write(text)


In [13]:
import json

# Load your data from a file
with open("/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/finetune_qa_good_results.json", "r") as f:
    data = json.load(f)

# Convert your data to the SQuAD format
squad_data = {
    "data": [
        {
            "paragraphs": [
                {
                    "context": d["context"],
                    "qas": [
                        {
                            "question": d["question"],
                            "answers": [
                                {
                                    "text": d["answer"],
                                    "answer_start": d["context"].find(d["answer"])
                                }
                            ]
                        }
                    ]
                }
            ]
        } for d in data
    ]
}

# Save the converted data to a file
with open("squad_finetune_qa_good_results.json", "w") as f:
    json.dump(squad_data, f)


In [1]:
import json

def assign_ids(dataset):
    for i, d in enumerate(dataset['data']):
        for j, p in enumerate(d['paragraphs']):
            dataset['data'][i]['paragraphs'][j]['id'] = f"{i}_{j}"
    return dataset

# Load the first JSON file
with open("/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/squad_finetune_qa_generated_results.json", "r") as f:
    dataset1 = json.load(f)

# Assign unique IDs to each context in the first JSON file
dataset1 = assign_ids(dataset1)

# Load the second JSON file
with open("/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/squad_finetune_qa_good_results.json", "r") as f:
    dataset2 = json.load(f)

# Assign unique IDs to each context in the second JSON file, matching the ID with the corresponding context in the first JSON file
for i, d in enumerate(dataset2['data']):
    for j, p in enumerate(d['paragraphs']):
        context = dataset2['data'][i]['paragraphs'][j]['context']
        for k, dp in enumerate(dataset1['data']):
            for l, pp in enumerate(dp['paragraphs']):
                if pp['context'] == context:
                    dataset2['data'][i]['paragraphs'][j]['id'] = f"{k}_{l}"
                    break

# Save the updated JSON files
with open("/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/squad_finetune_qa_generated_results.json", "w") as f:
    json.dump(dataset1, f)

with open("/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/squad_finetune_qa_good_results.json", "w") as f:
    json.dump(dataset2, f)


In [19]:
import json

with open("/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/squad_finetune_qa_generated_results.json", "r") as f1, open("/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/squad_finetune_qa_good_results.json", "r") as f2:
    data1 = json.load(f1)
    data2 = json.load(f2)

# Create a set of all contexts in data1
contexts1 = {p['context'] for d in data1['data'] for p in d['paragraphs']}

# Create a set of all contexts in data2
contexts2 = {p['context'] for d in data2['data'] for p in d['paragraphs']}

# Find the contexts in data1 that are not in data2
missing_contexts = contexts1 - contexts2

# Print the missing contexts
for context in missing_contexts:
    print(context)


In [2]:
import json

def read_json_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def write_json_file(data, filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def generate_qa_pairs(data):
    qa_pairs = []
    for paragraph in data['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            answer = qa['answers'][0]['text']
            qa_id = qa['id']
            qa_pairs.append((context, question, answer, qa_id))
    return qa_pairs

def update_ids(qa_pairs, qa_id_map):
    for qa in qa_pairs:
        old_qa_id = qa[3]
        new_qa_id = qa_id_map[str(old_qa_id)]
        qa_id_map[str(old_qa_id)] = new_qa_id
        qa[3] = new_qa_id

def main():
    filepath1 = '/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/output_new_squad_finetune_qa_generated_results.json'
    filepath2 = '/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/output_squad_finetune_qa_generated_results.json'
    data1 = read_json_file(filepath1)
    print(data1.keys())
    data2 = read_json_file(filepath2)
    print(data2.keys())
    qa_pairs1 = generate_qa_pairs(data1)
    qa_pairs2 = generate_qa_pairs(data2)
    qa_id_map = {}
    for i, qa in enumerate(qa_pairs1 + qa_pairs2):
        qa_id_map[str(qa[3])] = i
    update_ids(qa_pairs1, qa_id_map)
    update_ids(qa_pairs2, qa_id_map)
    write_json_file(data1, filepath1)
    write_json_file(data2, filepath2)

In [4]:
import json

def read_json_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def write_json_file(data, filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def generate_qa_pairs(data):
    qa_pairs = []
    for paragraph in data['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            qa['id'] = None  # Clear the ID
            question = qa['question']
            answer = qa['answers'][0]['text']
            qa_pairs.append((context, question, answer, None))
    return qa_pairs


def update_ids(qa_pairs, qa_id_map):
    for qa in qa_pairs:
        context_id, question, old_qa_id = qa[0], qa[2], qa[4] # modified
        new_qa_id = qa_id_map.get((context_id, question, old_qa_id), len(qa_id_map))
        qa_id_map[(context_id, question, old_qa_id)] = new_qa_id
        qa[4] = new_qa_id

def main():
    filepath1 = '/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/output_new_squad_finetune_qa_generated_results.json'
    filepath2 = '/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/output_squad_finetune_qa_generated_results.json'
    data1 = read_json_file(filepath1)
    print(data1.keys())
    data2 = read_json_file(filepath2)
    print(data2.keys())
    qa_pairs1 = generate_qa_pairs(data1)
    qa_pairs2 = generate_qa_pairs(data2)
    qa_id_map = {}
    for i, qa in enumerate(qa_pairs1 + qa_pairs2):
        context_id, question, qa_id = qa[0], qa[2], qa[4] # modified
        qa_id_map[(context_id, question, qa_id)] = i # modified
    update_ids(qa_pairs1, qa_id_map)
    update_ids(qa_pairs2, qa_id_map)
    write_json_file(data1, filepath1)
    write_json_file(data2, filepath2)


In [5]:
import json

# Load the first JSON file
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/output_squad_finetune_qa_generated_results.json') as f:
    data1 = json.load(f)

# Load the second JSON file
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/output_new_squad_finetune_qa_generated_results.json') as f:
    data2 = json.load(f)

# Loop through the paragraphs of the first JSON file
for p1 in data1['data'][0]['paragraphs']:
    context1 = p1['context']
    qas1 = p1['qas']

    # Loop through the qas of the first JSON file
    for qa1 in qas1:
        question1 = qa1['question']
        id1 = qa1['id']

        # Loop through the paragraphs of the second JSON file
        for p2 in data2['data'][0]['paragraphs']:
            context2 = p2['context']
            qas2 = p2['qas']

            # Check if the contexts match
            if context1 == context2:
                # Loop through the qas of the second JSON file
                for qa2 in qas2:
                    question2 = qa2['question']

                    # Check if the questions match
                    if question1 == question2:
                        qa2['id'] = id1

# Save the updated second JSON file
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/output_new_squad_finetune_qa_generated_results.json', 'w') as f:
    json.dump(data2, f)


In [5]:
import json
import subprocess

# load ground truth data
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/output_new_squad_finetune_qa_generated_results.json', 'r') as f:
    ground_truth = json.load(f)

# load predicted data
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/output_squad_finetune_qa_generated_results.json', 'r') as f:
    predicted = json.load(f)

# save predicted data in SQuAD format
data = []
for article in predicted['data']:
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        qas = []
        for qa in paragraph['qas']:
            qas_id = qa['id']
            question = qa['question']
            answers = [{'text': answer['text'], 'answer_start': answer['answer_start']} for answer in qa['answers']]
            qas.append({'id': qas_id, 'question': question, 'answers': answers})
        data.append({'paragraphs': [{'context': context, 'qas': qas}]})

with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/output_squad_finetune_qa_generated_results.json', 'w') as f:
    json.dump({'version': '1.1', 'data': data}, f)

# evaluate
cmd = ['python', 'evaluate-v1.1.py', '/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/output_new_squad_finetune_qa_generated_results.json', '/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/output_squad_finetune_qa_generated_results.json']
result = subprocess.run(cmd, stdout=subprocess.PIPE)

print(result.stdout.decode('utf-8'))

{"exact_match": 0.0, "f1": 0.0}



Evaluation expects v-1.1, but got dataset with v-v-1.1
Unanswered question 1 will receive score 0.
Unanswered question 2 will receive score 0.
Unanswered question 3 will receive score 0.
Unanswered question 4 will receive score 0.
Unanswered question 5 will receive score 0.
Unanswered question 6 will receive score 0.
Unanswered question 7 will receive score 0.
Unanswered question 8 will receive score 0.
Unanswered question 9 will receive score 0.
Unanswered question 10 will receive score 0.
Unanswered question 11 will receive score 0.
Unanswered question 12 will receive score 0.
Unanswered question 13 will receive score 0.
Unanswered question 14 will receive score 0.
Unanswered question 15 will receive score 0.
Unanswered question 16 will receive score 0.
Unanswered question 17 will receive score 0.
Unanswered question 18 will receive score 0.
Unanswered question 19 will receive score 0.
Unanswered question 20 will receive score 0.
Unanswered question 21 will receive score 0.
Unanswere

In [6]:
import json
import sklearn
from sklearn import metrics 
from sklearn.metrics import precision_recall_fscore_support

# Load ground truth data
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/output_new_squad_finetune_qa_generated_results.json', 'r') as f:
    ground_truth = json.load(f)

# Load predicted data
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/output_squad_finetune_qa_generated_results.json', 'r') as f:
    predicted = json.load(f)

# Flatten ground truth and predicted data
y_true = []
y_pred = []
for article in ground_truth['data']:
    for paragraph in article['paragraphs']:
        for qa in paragraph['qas']:
            qa_id = qa['id']
            if qa_id not in predicted:
                print(f"Missing id {qa_id} in predicted data")
            else:
                y_true.append(qa['answers'][0]['text'])  # Assume only one answer per question
                y_pred.append(predicted[qa_id][0])  # Assume only one answer per question

# Calculate precision, recall, and F1 score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

print('Precision:', precision)
print('Recall:', recall)
print('F1 score:', f1_score)

# Print out the IDs in the ground truth and predicted data
gt_ids = set()
pred_ids = set()
for article in ground_truth['data']:
    for paragraph in article['paragraphs']:
        for qa in paragraph['qas']:
            qa_id = qa['id']
            gt_ids.add(qa_id)
for qa_id in predicted:
    pred_ids.add(qa_id)
print(f"IDs in ground truth: {gt_ids}")
print(f"IDs in predicted: {pred_ids}")



Missing id 1 in predicted data
Missing id 2 in predicted data
Missing id 3 in predicted data
Missing id 4 in predicted data
Missing id 5 in predicted data
Missing id 6 in predicted data
Missing id 7 in predicted data
Missing id 8 in predicted data
Missing id 9 in predicted data
Missing id 10 in predicted data
Missing id 11 in predicted data
Missing id 12 in predicted data
Missing id 13 in predicted data
Missing id 14 in predicted data
Missing id 15 in predicted data
Missing id 16 in predicted data
Missing id 17 in predicted data
Missing id 18 in predicted data
Missing id 19 in predicted data
Missing id 20 in predicted data
Missing id 21 in predicted data
Missing id 22 in predicted data
Missing id 23 in predicted data
Missing id 24 in predicted data
Missing id 25 in predicted data
Missing id 26 in predicted data
Missing id 27 in predicted data
Missing id 28 in predicted data
Missing id 29 in predicted data
Missing id 30 in predicted data
Missing id 31 in predicted data
Missing id 32 in 

In [5]:
import json

# Load the ground-truth and predicted files
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/output_new_squad_finetune_qa_generated_results.json') as f:
    gt = json.load(f)
    
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/output_squad_finetune_qa_generated_results.json') as f:
    pred = json.load(f)

# Extract the question ids from the ground-truth file
gt_ids = []
for article in gt['data']:
    for paragraph in article['paragraphs']:
        for qa in paragraph['qas']:
            gt_ids.append(qa['id'])

# Extract the question ids from the predicted file
pred_ids = []
for article in pred['data']:
    for paragraph in article['paragraphs']:
        for qa in paragraph['qas']:
            pred_ids.append(qa['id'])

# Check if the ids are consistent between the ground-truth and predicted files
print(set(gt_ids) == set(pred_ids))


True


In [7]:
import json
import sklearn
from sklearn.metrics import precision_recall_fscore_support

# Load ground truth data
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/output_new_squad_finetune_qa_generated_results.json', 'r') as f:
    ground_truth = json.load(f)

# Load predicted data
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/output_squad_finetune_qa_generated_results.json', 'r') as f:
    predicted = json.load(f)

# Flatten ground truth and predicted data
y_true = []
y_pred = []
confidence_scores = []
for article in ground_truth['data']:
    for paragraph in article['paragraphs']:
        for qa in paragraph['qas']:
            qa_id = qa['id']
            if qa_id not in predicted:
                print(f"Missing id {qa_id} in predicted data")
            else:
                y_true.append(qa['answers'][0]['text'])  # Assume only one answer per question
                pred_answer = predicted[qa_id][0]['text']  # Assume only one answer per question
                confidence_scores.append(predicted[qa_id][0]['score'])  # Get confidence score for the predicted answer
                if pred_answer == "":
                    y_pred.append("No Answer")  # Treat empty answer as incorrect
                else:
                    y_pred.append(pred_answer)

# Set a threshold for confidence scores
threshold = 0.5

# Classify predicted answers as correct or incorrect based on the threshold
y_pred_binary = [1 if score >= threshold else 0 for score in confidence_scores]

# Calculate precision, recall, and F1 score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

print('Precision:', precision)
print('Recall:', recall)
print('F1 score:', f1_score)


Missing id 1 in predicted data
Missing id 2 in predicted data
Missing id 3 in predicted data
Missing id 4 in predicted data
Missing id 5 in predicted data
Missing id 6 in predicted data
Missing id 7 in predicted data
Missing id 8 in predicted data
Missing id 9 in predicted data
Missing id 10 in predicted data
Missing id 11 in predicted data
Missing id 12 in predicted data
Missing id 13 in predicted data
Missing id 14 in predicted data
Missing id 15 in predicted data
Missing id 16 in predicted data
Missing id 17 in predicted data
Missing id 18 in predicted data
Missing id 19 in predicted data
Missing id 20 in predicted data
Missing id 21 in predicted data
Missing id 22 in predicted data
Missing id 23 in predicted data
Missing id 24 in predicted data
Missing id 25 in predicted data
Missing id 26 in predicted data
Missing id 27 in predicted data
Missing id 28 in predicted data
Missing id 29 in predicted data
Missing id 30 in predicted data
Missing id 31 in predicted data
Missing id 32 in 

In [9]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from datasets import load_dataset, load_metric

# Load the SQuAD v2.0 dataset
dataset = load_dataset("squad_v2")

# Load the pre-trained model and tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",           # Output directory
    evaluation_strategy="steps",      # Evaluation strategy to adopt during training
    eval_steps=500,                   # Evaluation and Save happens every eval_steps
    save_total_limit=2,               # Only last 2 models are saved. Older ones are deleted.
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    logging_dir="./logs",
    logging_steps=1000,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
)

# Define the metric to use for evaluation
metric = load_metric("squad_v2")

# Define the function to preprocess the dataset
def preprocess_function(examples):
    # Tokenize the inputs and targets
    inputs = tokenizer(examples["question"], examples["context"], truncation=True)
    targets = {
        "start_positions": examples["answers"]["answer_start"],
        "end_positions": examples["answers"]["answer_start"] + len(examples["answers"]["text"]) - 1,
    }
    # Return the preprocessed examples
    return {"input_ids": inputs["input_ids"], "attention_mask": inputs["attention_mask"], "start_positions": targets["start_positions"], "end_positions": targets["end_positions"]}

# Preprocess the dataset
dataset = dataset.map(preprocess_function, batched=True)

# Define the function to compute the metrics during training
def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    predictions = predictions.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Instantiate the Trainer class and train the model
trainer = Trainer(
    model=model, args=training_args, train_dataset=dataset["train"], eval_dataset=dataset["validation"], compute_metrics=compute_metrics
)
trainer.train()


Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 1119.82it/s]


Dataset squad_v2 downloaded and prepared to /home/ssarrouf/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 359.62it/s]
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_out

TypeError: list indices must be integers or slices, not str

In [25]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import numpy as np
import torch
import json

class Example:
    def __init__(self, question, context):
        self.question = question
        self.context = context


# Load the fine-tuned model checkpoint
model_path = "/home/ssarrouf/Documents/qa_output/checkpoint-10000/"
model = AutoModelForQuestionAnswering.from_pretrained(model_path)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

# Load the SQuAD validation set
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationBERT/dev-v2.0.json', 'r') as f:
    squad = json.load(f)

def create_example(question, context):
    return Example(question, context)


def tokenize_example(example):
    # Tokenize the inputs
    inputs = tokenizer.encode_plus(
        example.question,
        example.context,
        add_special_tokens=True,
        return_token_type_ids=True,
        return_attention_mask=True,
        truncation_strategy='only_second',
        max_length=512
    )
    # Get the input_ids and segment_ids
    input_ids = inputs['input_ids']
    segment_ids = inputs['token_type_ids']

    print(input_ids)
    print(segment_ids)
    return input_ids, segment_ids


def answer_question(question, answer_text):
    # Create input example
    example = create_example(question, answer_text)
    print("example question:", example.question)
    print("example context:", example.context)

    # Tokenize the example
    input_ids, segment_ids = tokenize_example(example)

    # Get the start and end scores from the model
    
    start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))

    # Convert start and end scores to tensors
    start_scores = np.array(start_scores)
    print(start_scores.shape)
    start_scores_tensor = torch.tensor(start_scores.astype(float))
    #start_scores = torch.tensor(start_scores)
    #end_scores = torch.tensor(end_scores)
    end_scores = np.array(end_scores)
    end_scores_tensor = torch.tensor(end_scores.astype(float))

    # Find the tokens with the highest `start` and `end` scores
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        else:
            answer += ' ' + tokens[i]

    return answer


# Evaluate the model on the SQuAD validation set
correct_answers = 0
for article in squad['data']:
    for paragraph in article['paragraphs']:
        for qa in paragraph['qas']:
            question = qa['question']
            context = paragraph['context']
            predicted_answer = answer_question(question, context)
            for answer in qa['answers']:
                if predicted_answer.lower() == answer['text'].lower():
                    correct_answers += 1
                    break

total_questions = len(squad['data'])
accuracy = correct_answers / total_questions

print(f"Model accuracy on SQuAD validation set: {accuracy}")


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


example question: In what country is Normandy located?
example context: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
[101, 1999, 2054, 2406, 2003, 13298, 2284, 1029, 102, 1996, 5879, 2015, 1006, 5879, 1024, 2053, 3126, 2386, 5104, 1025, 2413, 1024, 5879, 5104, 1025, 3763, 1024, 5879, 3490, 1007, 2020,

ValueError: could not convert string to float: 'start_logits'

In [34]:
import torch
from transformers import BertTokenizer, BertForQuestionAnswering

# Load the fine-tuned BERT model
model_path = '/home/ssarrouf/Documents/qa_output/checkpoint-10000/pytorch_model.bin'
state_dict = torch.load(model_path, map_location=torch.device('cpu'))

# Initialize a new instance of the BERT model and load the state dict
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
model.load_state_dict(state_dict)
model.eval()

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define the input text
context = "My favorite food is pizza. I could eat pizza all day, every day!"
question = "What is my favorite food?"

# Tokenize the input text
inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors='pt')

# Get the start and end logits for the answer span
start_logits, end_logits = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])

# Find the tokens with the highest start and end logits
start_index = torch.argmax(start_logits)
end_index = torch.argmax(end_logits)
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
answer = tokens[start_index:end_index+1]
answer = tokenizer.convert_tokens_to_string(answer)

print(answer)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

RuntimeError: Error(s) in loading state_dict for BertForQuestionAnswering:
	size mismatch for bert.embeddings.word_embeddings.weight: copying a param with shape torch.Size([28996, 768]) from checkpoint, the shape in current model is torch.Size([30522, 768]).

In [32]:
# Import necessary libraries
import json
import torch
from transformers import BertTokenizer, BertForQuestionAnswering
from tqdm import tqdm
from sklearn.metrics import f1_score, precision_score, recall_score

# Load the fine-tuned model's state dict
model_path = '/home/ssarrouf/Documents/qa_output/checkpoint-10000/pytorch_model.bin'
state_dict = torch.load(model_path, map_location=torch.device('cpu'))

# Initialize a new instance of the BERT model and load the state dict
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
model.load_state_dict(state_dict)
model.eval()

# Load the SQuAD dev-v2.0 dataset
data_path = '/home/ssarrouf/Documents/GitHub/WaterRemediationBERT/dev-v2.0.json'
with open(data_path, 'r') as f:
    squad_dev = json.load(f)['data']

# Tokenize the input and get the predicted answer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
predictions = []
true_answers = []
for article in tqdm(squad_dev):
    for paragraph in article['paragraphs']:
        for qa in paragraph['qas']:
            question = qa['question']
            context = paragraph['context']
            input_ids = tokenizer.encode(question, context)
            start_scores, end_scores = model(torch.tensor([input_ids]))
            start_index = torch.argmax(start_scores)
            end_index = torch.argmax(end_scores)
            predicted_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[start_index:end_index+1]))
            predictions.append(predicted_answer)
            true_answers.append(qa['answers'][0]['text'])

# Calculate and print the evaluation metrics
f1 = f1_score(true_answers, predictions, average='weighted')
precision = precision_score(true_answers, predictions, average='weighted')
recall = recall_score(true_answers, predictions, average='weighted')

print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

RuntimeError: Error(s) in loading state_dict for BertForQuestionAnswering:
	size mismatch for bert.embeddings.word_embeddings.weight: copying a param with shape torch.Size([28996, 768]) from checkpoint, the shape in current model is torch.Size([30522, 768]).

In [37]:
# Import necessary libraries
import json
import torch
from transformers import BertTokenizer, BertForQuestionAnswering
from tqdm import tqdm
from sklearn.metrics import f1_score, precision_score, recall_score

# Load the fine-tuned model's state dict
model_path = '/home/ssarrouf/Documents/qa_output/checkpoint-10000/pytorch_model.bin'
state_dict = torch.load(model_path, map_location=torch.device('cpu'))

# Initialize a new instance of the BERT model with the same architecture as the pre-trained model
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

# Modify the weights of the pre-trained model to match the architecture of the new model
model_dict = model.state_dict()
for name, param in state_dict.items():
    if name.startswith('bert.'):
        name = '.'.join(name.split('.')[1:])  # Remove the "bert." prefix from the state dict keys
        if name.endswith('.gamma'):
            name = name[:-5] + '.weight'  # Replace ".gamma" with ".weight" in layer normalization parameters
        elif name.endswith('.beta'):
            name = name[:-4] + '.bias'  # Replace ".beta" with ".bias" in layer normalization parameters
        if name in model_dict:
            if isinstance(param, torch.nn.Parameter):
                param = param.data
            if model_dict[name].shape == param.shape:
                model_dict[name] = param
            else:
                print(f'Parameter shape mismatch for "{name}": pre-trained ({param.shape}) vs. new model ({model_dict[name].shape})')
        else:
            print(f'Unrecognized parameter "{name}" in the pre-trained model')
model.load_state_dict(model_dict)
model.eval()

# Load the SQuAD dev-v2.0 dataset
data_path = '/home/ssarrouf/Documents/GitHub/WaterRemediationBERT/dev-v2.0.json'
with open(data_path, 'r') as f:
    squad_dev = json.load(f)['data']

# Tokenize the input and get the predicted answer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
predictions = []
true_answers = []
for article in tqdm(squad_dev):
    for paragraph in article['paragraphs']:
        for qa in paragraph['qas']:
            question = qa['question']
            context = paragraph['context']
            input_ids = tokenizer.encode(question, context)
            start_scores, end_scores = model(torch.tensor([input_ids]))
            start_index = torch.argmax(start_scores)
            end_index = torch.argmax(end_scores)
            predicted_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[start_index:end_index+1]))
            predictions.append(predicted_answer)
            true_answers.append(qa['answers'][0]['text'])

# Calculate and print the evaluation metrics
f1 = f1_score(true_answers, predictions, average='weighted')
precision = precision_score(true_answers, predictions, average='weighted')
recall = recall_score(true_answers, predictions, average='weighted')

print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

Unrecognized parameter "embeddings.position_ids" in the pre-trained model
Unrecognized parameter "embeddings.word_embeddings.weight" in the pre-trained model
Unrecognized parameter "embeddings.position_embeddings.weight" in the pre-trained model
Unrecognized parameter "embeddings.token_type_embeddings.weight" in the pre-trained model
Unrecognized parameter "embeddings.LayerNorm.weight" in the pre-trained model
Unrecognized parameter "embeddings.LayerNorm.bias" in the pre-trained model
Unrecognized parameter "encoder.layer.0.attention.self.query.weight" in the pre-trained model
Unrecognized parameter "encoder.layer.0.attention.self.query.bias" in the pre-trained model
Unrecognized parameter "encoder.layer.0.attention.self.key.weight" in the pre-trained model
Unrecognized parameter "encoder.layer.0.attention.self.key.bias" in the pre-trained model
Unrecognized parameter "encoder.layer.0.attention.self.value.weight" in the pre-trained model
Unrecognized parameter "encoder.layer.0.attentio

  0%|          | 0/35 [00:01<?, ?it/s]


TypeError: argmax(): argument 'input' (position 1) must be Tensor, not str

In [39]:
import json

# load the new data from file
with open("/home/ssarrouf/Documents/GitHub/qa_validation_dataset.json", "r") as f:
    new_data = json.load(f)

# load the existing data from file
with open("/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/output_new_squad_finetune_qa_generated_results.json", "r") as f:
    existing_data = json.load(f)

# append the new data to the existing data
for item in new_data["data"]:
    for paragraph in item["paragraphs"]:
        paragraph["qas"][0]["id"] = str(int(paragraph["qas"][0]["id"].split("-")[-1]) + 1)
    existing_data["data"].append(item)

# save the updated data to file
with open("updated_data.json", "w") as f:
    json.dump(existing_data, f, indent=2)


In [41]:
import json

# Load the JSON data from file
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/updated_data.json') as f:
    data = json.load(f)

# Find the index of the item with id "241"
index = None
for i, item in enumerate(data['data']):
    if 'id' in item and item['id'] == '240':
        index = i
        break

# Update the IDs of subsequent items
if index is not None:
    for item in data['data'][index:]:
        if 'id' in item:
            item['id'] = str(int(item['id']) + 1)

# Save the updated data back to file
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/updated_data.json', 'w') as f:
    json.dump(data, f, indent=2)


In [26]:
import json
import re

def reset_answer_start(data):
    for elem in data["data"]:
        paragraphs = elem["paragraphs"]
        for paragraph in paragraphs:
            context = paragraph["context"]
            modified_context = re.sub(' +', ' ', context)
            modified_context = modified_context.lower()
            qas = paragraph["qas"]
            for qa in qas:
                answer_text = qa["answers"][0]["text"]
                answer_text = re.sub(' +', ' ', answer_text)
                answer_text = answer_text.lower()
                modified_answer_text = ''.join(c for c in answer_text if c.isalpha())
                modified_context_text = ''.join(c for c in modified_context if c.isalpha())
                match = re.search(re.escape(modified_answer_text), modified_context_text)
                if match:
                    original_answer_start = match.start()
                    # update answer_start to the index in the original context, accounting for removed spaces
                    modified_answer_start = len(answer_text) - len(answer_text[:original_answer_start].replace(' ', ''))
                    qa["answers"][0]["answer_start"] = context.find(answer_text, modified_answer_start)


# load the JSON file
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/updated_data_3_generated_predictions.json', 'r') as f:
    data = json.load(f)

# reset answer_start values if needed
reset_answer_start(data)

# write the modified data back to a file
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/updated_data_3_generated_predictions.json', 'w') as f:
    json.dump(data, f, indent=2)


In [39]:
import json
import re
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/updated_data_3_generated_predictions.json', 'r') as f:
    data = json.load(f)

for elem in data["data"]:
    paragraphs = elem["paragraphs"]
    for paragraph in paragraphs:
        context = paragraph["context"]
        modified_context = re.sub(' +', ' ', context)
        modified_context = modified_context.lower()
        qas = paragraph["qas"]
        for qa in qas:
            answer_start = qa["answers"][0]["answer_start"]
            if answer_start == -1:
                answer_text = qa["answers"][0]["text"].lower().strip().replace("  ", " ")
                match = re.search(re.escape(answer_text), modified_context)
                if match is not None:
                    qa["answers"][0]["answer_start"] = match.start()

# write the modified data back to a file
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/updated_data_3_generated_predictions.json', 'w') as f:
    json.dump(data, f, indent=2)

In [9]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np

# Create a SummaryWriter object with a log directory
writer = SummaryWriter('/home/ssarrouf/Documents/qa_finetune_output_2')

for n_iter in range(100):
    # Log the training and testing loss and accuracy
    writer.add_scalar('Loss/train', np.random.random(), n_iter)
    writer.add_scalar('Loss/test', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test', np.random.random(), n_iter)

# Close the SummaryWriter object
writer.close()


In [10]:
%load_ext tensorboard
%tensorboard --logdir /home/ssarrouf/Documents/qa_finetune_output_2 --port 6007


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [8]:
killall tensorboard

SyntaxError: invalid syntax (4231820316.py, line 1)

In [42]:
import json
from datasets import load_metric
import uuid

# Load the ground truth and actual prediction JSON files
with open("/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/updated_data_3.json", "r", encoding="utf-8") as f:
    ground_truth = json.load(f)


with open("/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/updated_data_3_generated_predictions.json", "r", encoding="utf-8") as f:
    predictions = json.load(f)

for i in range(len(predictions)):
    if "id" not in predictions[i]:
        predictions[i]["id"] = str(uuid.uuid4())
    if "id" not in ground_truth[i]:
        ground_truth[i]["id"] = str(uuid.uuid4())

matching_ids = set([qa["id"] for ex in predictions["data"] for para in ex["paragraphs"] for qa in para["qas"]])
for qa in ground_truth["data"]:
    if "qas" in qa:
        for q in qa["qas"]:
            if "id" in q and q["id"] not in matching_ids:
                print("Question ID not found in predictions:", q["id"])
ground_truth = [ex for ex in ground_truth["data"] for para in ex["paragraphs"] for qa in para["qas"] if "id" in qa and qa["id"] in matching_ids]
predictions = [ex for ex in predictions["data"] for para in ex["paragraphs"] for qa in para["qas"] if "id" in qa and qa["id"] in matching_ids]

# Compute the evaluation results
# You can replace "metric_name" with the name of the metric you want to use, such as "squad" or "bertscore"
metric = load_metric("squad")
results = metric.compute(predictions=predictions, references=ground_truth)

# Print the evaluation results
print(results)


KeyError: 'id'

In [29]:
import json

with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/output_new_squad_finetune_qa_generated_results.json') as f:
    data = json.load(f)

output = {
    "version": "v2.0",
    "data": [
        {
            "title": data[0]["title"],
            "paragraphs": []
        }
    ]
}

for obj in data[0]["paragraphs"]:
    output["data"][0]["paragraphs"].append({
        "qas": obj["qas"],
        "context": obj["context"]
    })

with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/updated3_output_ground_truth.json', 'w') as f:
    json.dump(output, f)


KeyError: 0

In [32]:
import json

with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/output_new_squad_finetune_qa_generated_results.json') as f:
    data = json.load(f)

output = {
    "version": "v-1.1",
    "data": [
        {
            "title": "squad desired results",
            "paragraphs": []
        }
    ]
}

for obj in data:
    output["data"][0]["paragraphs"].append({
        "qas": obj["qas"],
        "context": obj["context"]
    })

with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/updated3_output_ground_truth.json', 'w') as f:
    json.dump(output, f)


TypeError: string indices must be integers

In [ ]:
import json
from batterydataextractor.doc.text import Paragraph

# Load the ground truth data
with open('ground_truth.json', 'r') as f:
    data = json.load(f)

# Extract predictions for each question and context in the ground truth data
for item in data['data']:
    for para in item['paragraphs']:
        for qas in para['qas']:
            # Get the question and context text
            question = qas['question']
            context = para['context']
            
            # Use Battery Data Extractor to generate predictions
            doc = Paragraph(context)
            doc.add_general_models([question], confidence_threshold=0.1, self_defined=True)
            
            # Add the predictions to the ground truth data
            qas['predicted_answers'] = doc.records[0].answers
            
# Write the updated ground truth data to a new file
with open('battery_data_predictions.json', 'w') as f:
    json.dump(data, f)


In [35]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Load the fine-tuned model and tokenizer
model_name = '/home/ssarrouf/Documents/qa_output/checkpoint-10000'  # replace with the name of your fine-tuned model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Load the contexts and questions from the input JSON file
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/updated_data_3.json') as f:
    data = json.load(f)

# Define the output JSON data
output_data = {
    "version": "v2.0",
    "data": []
}

# Loop through the data and generate predictions
for item in data['data']:
    title = item.get('title', '')
    paragraphs = item.get('paragraphs', [])
    output_paragraphs = []

    for paragraph in paragraphs:
        qas = paragraph.get('qas', [])
        context = paragraph.get('context', '')
        output_qas = []

        for qa in qas:
            id = qa.get('id', '')
            question = qa.get('question', '')

            # Tokenize the input
            inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")

            # Generate the predictions
            outputs = model(**inputs)
            start_scores, end_scores = outputs.start_logits, outputs.end_logits
            start_index = torch.argmax(start_scores)
            end_index = torch.argmax(end_scores)
            answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_index:end_index+1]))

            # Create the output QA data
            output_qa = {
                "id": id,
                "question": question,
                "answers": [
                    {
                        "text": answer,
                        "answer_start": context.find(answer)
                    }
                ]
            }

            output_qas.append(output_qa)

        # Create the output paragraph data
        output_paragraph = {
            "qas": output_qas,
            "context": context
        }

        output_paragraphs.append(output_paragraph)

    # Create the output data for this item
    output_item = {
        "title": title,
        "paragraphs": output_paragraphs
    }

    output_data['data'].append(output_item)

# Save the output data to a new JSON file
with open('output2_generated_predictions.json', 'w') as f:
    json.dump(output_data, f)


In [37]:
import json

# Load the data from the JSON file
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/updated_data.json', 'r') as f:
    data = json.load(f)


# Find the index of the first question with an ID greater than 240
start_index = None
for i in range(len(data['data'])):
    for j in range(len(data['data'][i]['paragraphs'][0]['qas'])):
        if int(data['data'][i]['paragraphs'][0]['qas'][j]['id']) > 240:
            start_index = (i, j)
            break
    if start_index:
        break

# Modify the IDs of the subsequent questions
current_id = 241
if start_index:
    for i in range(start_index[0], len(data['data'])):
        for j in range(start_index[1], len(data['data'][i]['paragraphs'][0]['qas'])):
            data['data'][i]['paragraphs'][0]['qas'][j]['id'] = str(current_id)
            current_id += 1


# Write the modified data back to the original JSON file
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/updated_data.json', 'w') as f:
    json.dump(data, f)


In [2]:
import json

# Load the JSON data
with open("/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/updated_data.json", "r") as f:
    data = json.load(f)

# Find the index of the first question with ID of 240
start_index = None
for i in range(len(data['data'])):
    for j in range(len(data['data'][i]['paragraphs'])):
        for k in range(len(data['data'][i]['paragraphs'][j]['qas'])):
            if data['data'][i]['paragraphs'][j]['qas'][k]['id'] == '240':
                start_index = (i, j, k)
                break
        if start_index:
            break
    if start_index:
        break

if start_index is None:
    print(f"No question with id 240 was found")
    exit()

# Modify the IDs of the subsequent questions
current_id = 241
for i in range(start_index[0], len(data['data'])):
    for j in range(start_index[1], len(data['data'][i]['paragraphs'])):
        for k in range(start_index[2], len(data['data'][i]['paragraphs'][j]['qas'])):
            data['data'][i]['paragraphs'][j]['qas'][k]['id'] = str(current_id)
            current_id += 1

# Write the modified data back to the original JSON file
with open("/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/updated_data_2.json", "w") as f:
    json.dump(data, f)


In [ ]:
data = json.loads(json_string)

# Get the starting ID for the new questions
starting_id = int(data["data"][0]["qas"][0]["id"]) + 1

# Loop through the contexts and questions to update the IDs
for i, paragraph in enumerate(data["data"]):
    for j, qa in enumerate(paragraph["qas"]):
        if qa["id"] != "239":
            qa["id"] = str(starting_id)
            starting_id += 1

# Convert the updated data back to JSON
updated_json_string = json.dumps(data)


In [6]:
import json

# Load the JSON data
with open("/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/output2_generated_predictions.json", "r") as f:
    data = json.load(f)

# Find the index of the question with ID of 239
start_index = None
for i in range(len(data['data'])):
    for j in range(len(data['data'][i]['paragraphs'])):
        for k in range(len(data['data'][i]['paragraphs'][j]['qas'])):
            if data['data'][i]['paragraphs'][j]['qas'][k]['id'] == '240':
                start_index = (i, j, k)
                break
        if start_index:
            break
    if start_index:
        break

if start_index is None:
    print(f"No question with id 239 was found")
    exit()

# Modify the IDs of the subsequent questions
current_id = 241
for i in range(start_index[0], len(data['data'])):
    for j in range(start_index[1], len(data['data'][i]['paragraphs'])):
        for k in range(start_index[2], len(data['data'][i]['paragraphs'][j]['qas'])):
            if data['data'][i]['paragraphs'][j]['qas'][k]['id'] != '240':
                data['data'][i]['paragraphs'][j]['qas'][k]['id'] = str(current_id)
                current_id += 1

# Write the modified data back to the original JSON file
with open("/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/updated_data_3_generated_predictions.json", "w") as f:
    json.dump(data, f)


In [1]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Load the SQuAD ground truth file
with open('/home/ssarrouf/Documents/GitHub/WaterRemediationParser/performance_evaluation/train-v1.1.json', 'r') as f:
    squad_data = json.load(f)

# Initialize an empty list for the predictions
predictions = []

# Load your tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("/home/ssarrouf/Documents/qa_output/checkpoint-10000")
model = AutoModelForQuestionAnswering.from_pretrained("/home/ssarrouf/Documents/qa_output/checkpoint-10000")

# Check the model's configuration
print(model.config)

# Loop over each article in the SQuAD ground truth file
for article in squad_data['data']:
    # Initialize an empty list for the paragraphs in the article
    paragraphs = []

    # Loop over each paragraph in the article
    for paragraph in article['paragraphs']:
        # Initialize an empty list for the QA pairs in the paragraph
        qas = []

        # Loop over each QA pair in the paragraph
        for qa in paragraph['qas']:
            # Tokenize the question and context
            question = qa['question']
            context = paragraph['context']
            inputs = tokenizer.encode_plus(question, context, return_tensors="pt")
            print(inputs)

            # Generate the model's prediction
            outputs = model(**inputs)
            start_scores, end_scores = outputs.start_logits, outputs.end_logits
            print(type(start_scores))
            print(type(end_scores))
            answer_start = torch.argmax(start_scores)
            answer_end = torch.argmax(end_scores) + 1


            # Get the tokens corresponding to the best start and end positions
            answer_tokens = inputs["input_ids"][0][answer_start:answer_end]
            answer = tokenizer.decode(answer_tokens)

            # Create the answer dictionary
            answer_dict = {
                'text': answer,
                'answer_start': answer_start.item()
            }

            # Create the QA dictionary
            qa_dict = {
                'question': qa['question'],
                'id': qa['id'],
                'answers': [answer_dict]
            }

            # Add the QA dictionary to the list of QA pairs
            qas.append(qa_dict)

        # Create the paragraph dictionary
        paragraph_dict = {
            'context': paragraph['context'],
            'qas': qas
        }

        # Add the paragraph dictionary to the list of paragraphs
        paragraphs.append(paragraph_dict)

    # Create the article dictionary
    article_dict = {
        'title': article['title'],
        'paragraphs': paragraphs
    }

    # Add the article dictionary to the list of articles
    predictions.append(article_dict)

# Save the predictions to a JSON file
with open('squad_train_v1.1_predictions.json', 'w') as f:
    json.dump({'data': predictions}, f)


2023-04-03 09:37:27.369632: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 09:37:28.230520: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using TensorFlow backend.


BertConfig {
  "_name_or_path": "/home/ssarrouf/Documents/qa_output/checkpoint-10000",
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

{'input_ids': tensor([[  101,  1106,  2292,  1225,  1103, 16946, 12477,  1616,  9273,  2845,
          1107,  8109,  1107, 25338,  2149,  4704,   175, 10555,   136,   102,
          6645,  1193,   117,  1103,  1278,  1144,   170,  5855, 14084,  1596,
          1959,   119, 10343, 

TypeError: argmax(): argument 'input' (position 1) must be Tensor, not str